# 02 - Silver Layer

Limpeza, transformação e validação dos dados.


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os


In [ ]:
# Carregar dados
df = pd.read_csv('data/bronze/dados_brutos.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M')

print(f"Dados carregados: {len(df):,} linhas")
df.head()


✓ Dados carregados: 541,909 linhas


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,data_ingestao,fonte_arquivo
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2025-10-31 19:38:51.638174,data.csv
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2025-10-31 19:38:51.638174,data.csv
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2025-10-31 19:38:51.638174,data.csv
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2025-10-31 19:38:51.638174,data.csv
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2025-10-31 19:38:51.638174,data.csv


## 1. Limpeza e Transformação


In [ ]:
# Remover duplicatas
df = df.drop_duplicates()

# Tratar nulos
df['CustomerID'] = df['CustomerID'].fillna(0).astype(int)
df['Description'] = df['Description'].fillna('PRODUTO SEM DESCRIÇÃO')

# Converter tipos
df['Country'] = df['Country'].astype('category')
df['StockCode'] = df['StockCode'].astype('category')

# Padronizar texto
df['Description'] = df['Description'].str.strip().str.upper()
df['Country'] = df['Country'].astype(str).str.strip()

# Remover preços negativos e criar colunas calculadas
df = df[df['UnitPrice'] >= 0]
df['eh_devolucao'] = df['Quantity'] < 0
df['valor_total'] = df['Quantity'] * df['UnitPrice']

print(f"Transformações aplicadas: {len(df):,} linhas")


✓ Transformações aplicadas: 536,639 linhas


## 2. Salvar Silver


In [ ]:
# Adicionar metadados e salvar
df['data_processamento'] = datetime.now()

os.makedirs('data/silver', exist_ok=True)
df.to_csv('data/silver/dados_limpos.csv', index=False)

print("Dados salvos na camada Silver")
print(f"  {df.shape[0]:,} linhas x {df.shape[1]} colunas")


✓ Dados salvos na camada Silver
  536,639 linhas x 13 colunas
